# INPUT AND FILTER STAGES

This program takes and converts the input and processes it through the filter stages for both, midget and parasol cells.

Make sure you've installed all the necessary modules.

In [ ]:
#!/bin/python
#THIS PART READS THE PIXELS OF A MOVIE AND APPLIES THE TEMPORAL FILTERS TO THE PROPOSED MODEL FOR MICROSACCADES. 
#THE OUTPUT ARE THE POTENTIAL VALUES FOR THE POISSON RATES.

'''
loading the video into list
-> first get all the pixels in a frame (grayscale)
-> afterwards calculate the values of each tempral filter at each time and store them in another list
which will be the basis for changing poisson rates
'''

import sys
import os
import glob
import pylab as pyl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import datetime
import itertools 
#import multiprocessing
from microsaccades_functions import *

#---------------------------------------------------------------------------------------------------LOAD-FRAMES
now = datetime.datetime.now()

pgf_with_rc_fonts = {"font.family": "serif","font.serif": [],"font.sans-serif": []}
plt.rcParams.update(pgf_with_rc_fonts)

Read input from command line

In [ ]:
sim_title = sys.argv[1]
handle_name = sys.argv[2]
fn = sys.argv[3]

Next, create a list of all the files (starting "second") in the respective repository. Using the cv2 module, the grayscale values for each pixel are stored for each time frame in the list `pixels4d`.

In [ ]:
frames = []
frame_number = int(fn)
os.chdir("video/img_input/" + sim_title)
for file in glob.glob("second*.png"):
    frames+=[file]
frames.sort()

f=cv2.imread(frames[0])
height, width = f.shape[:2]
dt = 1.
print height, width

#assign the all time all pixel list
pixels4d = [[[] for j in range(width)] for i in range(height)]

for file in frames:

    frame = cv2.imread(file)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #store 2D list in pixels4d 
    for i in range(height):
        for j in range(width):
            pixels4d[i][j]+=[float(gray[i,j])]
 
cv2.destroyAllWindows()

os.chdir("../../..")

In [ ]:
#----------------------------------------------------------------------------------------------------INITIALIZE
on_tau1 = 10. 
on_tau2 = 15. 
on_p = .65 

#set values for the spatial filters for the pixel
alpha = .1
beta = 3. #CSR
sigma = 1. # parasolic/midget cell ratio * sigma (=1)
px_midget_ratio = 1. #pixel to receptor ratio, needed for receptor distance/number
px_dist= 0.5 #in arcmin
par_m_ratio = 4. #2arcmin center field sigma_c

#spatial filter values determined
spat_filter_break_radius = 6 #filter radius in px, higher distances omitted

#subsequent values
midget_dist = px_midget_ratio #1, rest gone, since unit is 1px

Get the shape of the temporal filter in 1ms resolution for 200ms.

In [ ]:
temp_filter_on = tempFilter(200,dt,on_tau1,on_tau2,on_p)      

midget_height = int(height/(px_midget_ratio*0.866)) #hexagonal
midget_width = int(width/px_midget_ratio)

parasol_height = int(height/(par_m_ratio*0.866))
parasol_width = int(width/par_m_ratio)

#to store values after filter stages
midget_pixels4d = np.zeros(shape=(midget_height,midget_width,frame_number))
midget_pixels4d = [[[] for j in range(midget_width)] for i in range(midget_height)]

#grid structure for cells
grid=[[0],[0]]
m_grid=[[0],[0]]
pgrid=[[0],[0]]

midget_grid= [[(0,0) for j in range(midget_width)] for i in range(midget_height)]

#---------------------------------------------------------------------------------------MIDGETS-SPATIAL-FILTER 
print '---'
print 'start of midget part'

spat_filt_values = [[[] for j in range(midget_width)] for i in range(midget_height)]

### SPATIAL FILTERS - MIDGET CELLS

Now, two functions are created to calculate the spatially filtered values for the midget cells. 
`spatFilterPx()` calculates the weighted value for each pixel at distance dist depending on the position of cell. Distances further than `r_break` lead to filter values = 0.

In [ ]:
def spatFilterPx(center,kl,r_break,sigma,alpha,beta):

    klv_w = 0
    klv_h = 0
    new_kl0,new_kl1 = kl
    kl_pos = float(kl[0]),float(kl[1])
    
    dis_x = (center[0]-kl_pos[0])
    dis_y = (center[1]-kl_pos[1])
    
    dist=np.sqrt(dis_x*dis_x+dis_y*dis_y)

    if dist <= r_break:
        spat_filter_value = spatialFilter(dist,0,sigma,alpha,beta)
    else:
        spat_filter_value = 0.
    
    return spat_filter_value

`getSpatFilter(ij)` calculates the position of each cell `ij` at each time frame `q` and creates a list of input pixels that are close by in intervals $[i_{low};i_{ceil}]$, resp. $[j_{low};j_{ceil}]$. It then uses `spatFilterPx()` to add up the filter values for these pixels regarding the midget cell. `pixq` and `spat_filter_values` then get multiplied (with circular boundary conditions).

In [ ]:
def getSpatFilter(ij):
    
    if ij[0]%5==0:
        if ij[1]==0:
            print ij[0]
            
    pos_i = 0.866*ij[0]        #2*cos(30deg)*ij[0]  
    pos_j = ij[1]
    
    if ij[0]%2 == 0:
        pos_j += 0.5

    i_low = int(pos_i*px_midget_ratio-spat_filter_break_radius) # -> all j with dist < r
    i_ceil = int(pos_i*px_midget_ratio+spat_filter_break_radius)+1
    j_low = int(pos_j*px_midget_ratio-spat_filter_break_radius) # -> all j with dist < r
    j_ceil = int(pos_j*px_midget_ratio+spat_filter_break_radius)+1

    
    #(height, width)
    pos=(pos_i,pos_j)
    
    #if f==0:
    grid[0] += [ij[0]]
    grid[1] += [ij[1]]
    m_grid[0] += [pos_j]
    m_grid[1] += [pos_i]
    midget_grid[ij[0]][ij[1]] = (pos_j, pos_i) 
    
    #create list with numpy arrays here
    #define length + list beforehand
    
    mp=[0. for g in range(frame_number)]
    q_len = abs((j_ceil-j_low)*(i_ceil-i_low))
    pxiq = [] #0 for q in range(q_len)]
    
    for k in range(i_low,i_ceil):
        for l in range(j_low,j_ceil):
            spat_filt_values[ij[0]][ij[1]] += [spatFilterPx(pos,(k,l),spat_filter_break_radius,sigma,alpha,beta)]
            pxiq+=[pixels4d[(height+k)%height][(width+l)%width]]

    #spatial filters can allow negative potentials
    for q in xrange(q_len):
        w = spat_filt_values[ij[0]][ij[1]][q]
        muli = [o/255.*w for o in pxiq[q]]
        mp = list(itertools.imap(lambda w,z: w+z, muli, mp))
        
    midget_pixels4d[ij[0]][ij[1]] = mp

This function iterates through all midget cells, until all spatially filtered values for each midget cell at each time frame are obtained.

In [ ]:
map(lambda x: getSpatFilter(x), itertools.product(range(midget_height),range(midget_width)))

### TEMPORAL FILTERS - MIDGET CELLS

In [ ]:
#output frame for the filtered values at the end
#including negative values
temp_filter_vals  = np.zeros(shape=(midget_height,midget_width,frame_number))
#setting negative values to zero
temp_filter_vals_on  = np.zeros(shape=(midget_height,midget_width,frame_number))

The next step is the temporal convolution for each time frame and midget cell.

In [ ]:
#--------------------------------------------------------------------------------------MIDGETS-TEMPORAL-FILTERS
print '---'
print 'start of temporal part'
#now apply the temporal filters: output are two lists for on/off fields
for i in range(midget_height):
    if i%5==0:
        print i
    for j in range(midget_width):
        temp_midget_px4d = []
        pop = temp_midget_px4d.pop
        for f in range(frame_number):
            #restriction to current 200ms frames
            temp_midget_px4d.insert(0, float(midget_pixels4d[i][j][f]))
            if f > 200:
                pop()
            #add a new entry to the time list of pixel i,j
            #perform the convolution
            trs = sum(itertools.imap(lambda x,y: x*y, temp_midget_px4d, temp_filter_on))
            temp_filter_vals_on[i][j][f] = trs if trs > 0 else 0
            temp_filter_vals[i][j][f] = trs
            #temp_filter_vals_off[i][j][f] = sum(itertools.imap(lambda x,y: x*y, temp_midget_px4d, temp_filter_off))


#convert to numpy arrays
m_output = np.asarray(temp_filter_vals)
m_output_on = np.asarray(temp_filter_vals_on)

### SPATIAL FILTER - PARASOL CELLS

In [ ]:
#--------------------------------------------------------------------------------------PARASOLS-SPATIAL-FILTERS

print '---'
print 'start of parasol part'
parasol_grid= [[(0,0) for j in range(parasol_width)] for i in range(parasol_height)]

#only rectified values
par_values = [[[0. for f in range(frame_number)] for j in range(parasol_width)] for i in range(parasol_height)] 
par_values_on = [[[0. for f in range(frame_number)] for j in range(parasol_width)] for i in range(parasol_height)] 
par_values_on_off = [[[0. for f in range(frame_number)] for j in range(parasol_width)] for i in range(parasol_height)] 

#spatially filtered values
p_spat_filt_values = [[[] for j in range(midget_width)] for i in range(midget_height)]

Again, we define two functions for the spatial filtering, following the structure for the midget cells.
First the function to calculate the filter weight.

In [ ]:
def spatFilterParasol(center,kl,r_break,sigma,alpha,beta):

    klv_w = 0
    klv_h = 0
    new_kl0,new_kl1 = kl
    
    kl_val = midget_grid[(midget_height+kl[0])%midget_height][(midget_width+kl[1])%midget_width]
    n_kl_h=kl_val[0]
    n_kl_w=kl_val[1]
    
    #if that seems strange: midget_grid is transposed in order to show it better in graphics etc.
    if kl[0] >= midget_height:
        n_kl_w=kl_val[1]+height
    if kl[0] < 0:
        n_kl_w=kl_val[1]-height
    if kl[1] >= midget_width:
        n_kl_h=kl_val[0]+width
    if kl[1] < 0:
        n_kl_h=kl_val[0]-width
    
    kl_pos=(n_kl_h,n_kl_w)
    
    dis_x = (center[0]-kl_pos[0])
    dis_y = (center[1]-kl_pos[1])
    
    dist=np.sqrt(dis_x*dis_x+dis_y*dis_y)

    if dist <= r_break:
        spat_filter_value = spatialFilter(dist,0,sigma,alpha,beta)
    else:
        spat_filter_value = 0.
    
    return spat_filter_value

The function for the filtered output for each frame and parasol cell.

In [ ]:
def getSpatFilterParasol(ij):
    
    if ij[0]%5==0:
        if ij[1]==0:
            print ij[0]
    
    pos_i = 0.866*par_m_ratio*ij[0]        #2*cos(30deg)
    pos_j = par_m_ratio*ij[1]
    
    if ij[0]%2 == 0:
        pos_j += par_m_ratio*0.5 

    move = (par_m_ratio)*spat_filter_break_radius/px_midget_ratio
    
    i_low = int(par_m_ratio*ij[0]-move/0.866) # -> all j with dist < r
    i_ceil = int(par_m_ratio*ij[0]+move/0.866)+1  
    j_low = int(par_m_ratio*ij[1]-move) # -> all j with dist < r
    j_ceil = int(par_m_ratio*ij[1]+move)+1
    
    pos = (pos_j, pos_i) 
    
    if f==0:
        pgrid[0] += [pos_j]
        pgrid[1] += [pos_i]
        parasol_grid[ij[0]][ij[1]] = pos 

        
    pp=[0. for g in range(frame_number)]
    q_len = abs((j_ceil-j_low)*(i_ceil-i_low))
    mgiq = [] #0 for q in range(q_len)]
    
    #next line is only to show the grid 
    #to_graph = [[0. for l in range(j_ceil-j_low)] for k in range(i_ceil-i_low)]
    for k in range(i_low,i_ceil):
        for l in range(j_low,j_ceil):
            p_spat_filt_values[ij[0]][ij[1]] += [spatFilterParasol(pos,(k,l),(par_m_ratio)*spat_filter_break_radius,par_m_ratio*sigma,alpha,beta)]
            mgiq += [temp_filter_vals_on[(midget_height+k)%midget_height][(midget_width+l)%midget_width]]
            #if ij[0]==0:
            #    if ij[1]==16:
            #        to_graph[k-i_low][l-j_low]=spatFilterParasol(pos,(k,l),(par_m_ratio)*spat_filter_break_radius,par_m_ratio*sigma,alpha,beta) 
    
    '''
    #leave this for possible further investigation later on
    if ij[0]==0:
        if ij[1]==16:
            fig = plt.figure(1)

            ax = fig.add_subplot(111)
            ax.set_title('parasolic filter')
            plt.imshow(to_graph, aspect='auto', interpolation='nearest')
            ax.set_aspect('equal')
            plt.axis('off')

            cax = fig.add_axes([0.,0.,1.,1.])
            cax.get_xaxis().set_visible(False)
            cax.get_yaxis().set_visible(False)
            cax.patch.set_alpha(0)
            cax.set_frame_on(False)
            
            plt.show()
            plt.close()
    '''
    
    for q in xrange(q_len):
        w = p_spat_filt_values[ij[0]][ij[1]][q]
        muli = [o*w for o in mgiq[q]]
        pp = list(itertools.imap(lambda w,z: w+z, muli, pp))
        
    par_values_on[ij[0]][ij[1]] = pp  

Map function terating through all cells.

In [ ]:
map(lambda x: getSpatFilterParasol(x), itertools.product(range(parasol_height), range(parasol_width)))

print 'parasols done'

Save grid positions for `ms_network.py` part.

In [ ]:
m_pos = np.asarray(midget_grid)
m_pos_data = open('/data/'+sim_title+'/m_pos_'+str(handle_name)+'.data','w+')
np.save(m_pos_data, m_pos)
m_pos_data.close()

p_pos = np.asarray(parasol_grid)
p_pos_data = open('/data/'+sim_title+'/p_pos_'+str(handle_name)+'.data','w+')
np.save(p_pos_data, p_pos)
p_pos_data.close()

Save filtered values and finish program.

In [ ]:
#--------------------------------------------------------------------------------------------------------OUTPUT
p_output = np.asarray(par_values)
p_output_on = np.asarray(par_values_on)

#---------------------------------------------------------------------------------------------------SAVE-OUTPUT
m_data = open('/home/schrader/Documents/microsaccades/data/'+sim_title+'/midget_rates_'+str(handle_name)+'.data','w+')
np.save(m_data, m_output)
m_data.close()

p_data = open('/home/schrader/Documents/microsaccades/data/'+sim_title+'/parasolic_rates_'+str(handle_name)+'.data','w+')
np.save(p_data, p_output)
p_data.close()

m_data = open('/home/schrader/Documents/microsaccades/data/'+sim_title+'/midget_rates_'+str(handle_name)+'_on.data','w+')
np.save(m_data, m_output_on)
m_data.close()

p_data = open('/home/schrader/Documents/microsaccades/data/'+sim_title+'/parasolic_rates_'+str(handle_name)+'_on.data','w+')
np.save(p_data, p_output_on)
p_data.close()

print '--- PROGRAM FINISHED ---'
sys.exit()